In [1]:
import os
import numpy as np
import pandas as pd

from sacremoses import MosesDetokenizer
from tqdm import tqdm


md = MosesDetokenizer(lang='en')
type_map = {
    "name": "Name",
    "eatType": "Eat type",
    "food": "Food",
    "customerRating": "Customer rating",
    "area": "Area",
    "familyFriendly": "Family friendly",
    "priceRange": "Price range",
    "near": "Near"
}

In [2]:
def read_from_file(fname, strip=True):
    with open(fname, "r") as f:
        if strip:
            return [line.strip() for line in f]
        else:
            return [line for line in f]

def postprocess_e2e_preds(preds):
    processed_preds = []
    for pred in tqdm(preds):
        processed_preds.append(md.detokenize(pred.replace("_", " ").split()))
    return processed_preds

def process_type_lines(type_lines):
    processed_types = []
    for type_line in type_lines:
        processed_types.append([type_map[type] for type in type_line.split()])
    return processed_types

def process_value(value):
        if value == "family_friendly":
            return "yes"
        elif value == "not_family_friendly":
            return "no"
        else:
            return value.replace("_", " ")

def process_value_lines(value_lines):
    processed_values = []
    for value_line in value_lines:
        processed_values.append([process_value(value) for value in value_line.split()])
    return processed_values


In [3]:
data_dir = "/projects/ogma2/users/andrewsi/control-data2text/DTG-SI/e2e_data/val"
baseline_preds_file = "/projects/ogma2/users/andrewsi/control-data2text/transformers/examples/seq2seq/exp/e2e/e2e_t5_small_01/checkpoint-10000/validation_preds.txt"
dtg_si_preds_file = "/projects/ogma2/users/andrewsi/control-data2text/DTG-SI/e2e_dtg_si/ckpt/hypos.step1200.val.txt"
our_preds_file = "/projects/ogma2/users/andrewsi/control-data2text/transformers/examples/seq2seq/exp/e2e/e2e_k3_t5_small_01/checkpoint-8295/validation_preds.txt"

x_types = process_type_lines(read_from_file(f"{data_dir}/x_type.valid.txt"))
x_values = process_value_lines(read_from_file(f"{data_dir}/x_value.valid.txt"))
y_refs = postprocess_e2e_preds(read_from_file(f"{data_dir}/y_ref.valid.txt"))

baseline_preds = postprocess_e2e_preds(read_from_file(baseline_preds_file))
dtg_si_preds = postprocess_e2e_preds(read_from_file(dtg_si_preds_file))
our_preds = postprocess_e2e_preds(read_from_file(our_preds_file))


100%|██████████| 6300/6300 [00:03<00:00, 1750.23it/s]


In [16]:
indices = [4653, 2659, 5128, 5810, 2976, 2096, 2063, 3507, 5783, 4778, 4105, 3671, 350, 4589, 5183, 2109, 4, 5656, 1779, 4169, 5105, 5988, 5960, 397, 1534, 6161, 5622, 2177, 3010, 706, 597, 6234, 612, 4963, 4255, 5463, 655, 269, 2937, 1603, 3943, 1850, 6176, 6233, 721, 732, 1420, 5103, 1781, 4899, 438, 5581, 6154, 3295, 45, 2990, 4125, 5830, 2053, 3034, 920, 158, 1180, 3630, 4695, 5541, 2502, 4436, 5527, 2059, 4587, 2562, 2561, 1861, 3419, 3684, 2164, 1523, 5139, 1008, 298, 6120, 1156, 1260, 5652, 388, 4814, 1301, 4531, 2076, 4406, 843, 24, 35, 1164, 968, 5951, 3865, 977, 637]

In [14]:
junxian = [4653, 2659, 5128, 5810, 2976, 2096, 2063, 3507, 5783, 4778, 4105, 3671, 350, 4589, 5183, 2109, 4, 5656, 1779, 4169, 5105, 5988, 5960, 397, 1534, 6161, 5622, 2177, 3010, 706, 597, 6234, 612, 4963, 4255, 5463, 655, 269, 2937, 1603, 3943, 1850, 6176, 6233, 721, 732, 1420, 5103, 1781, 4899]

junxian2 = [438, 5581, 6154, 3295, 45, 2990, 4125, 5830, 2053, 3034, 920, 158, 1180, 3630, 4695, 5541, 2502, 4436, 5527, 2059, 4587, 2562, 2561, 1861, 3419, 3684, 2164, 1523, 5139, 1008, 298, 6120, 1156, 1260, 5652, 388, 4814, 1301, 4531, 2076, 4406, 843, 24, 35, 1164, 968, 5951, 3865, 977, 637]

In [26]:
np.random.shuffle(indices)

In [33]:
indices[49]

4963

In [29]:
eval_sheet = []
# eval_sample = np.random.choice(len(x_type_lines), 100, replace=False)

for i in indices:
    eval_sheet.append(["Table", "", "Exemplar", "Generations", "Factual Consistency", "Style Embodiment", "Fluency", "ID", str(i)])
    gens = [baseline_preds[i], dtg_si_preds[i], our_preds[i]]
    gen_idx_perm = np.random.permutation(range(3))
    shuffled_gens = [gens[i] for i in gen_idx_perm]
    eval_sheet.append([x_types[i][0], x_values[i][0], y_refs[i], shuffled_gens[0], "", "", "", str(gen_idx_perm[0]), ""])
    eval_sheet.append([x_types[i][1], x_values[i][1], "", shuffled_gens[1], "", "", "", str(gen_idx_perm[1]), ""])
    eval_sheet.append([x_types[i][2], x_values[i][2], "", shuffled_gens[2], "", "", "", str(gen_idx_perm[2]), ""])
    for j in range(3, len(x_types[i])):
        eval_sheet.append([x_types[i][j], x_values[i][j], "", "", "", "", "", "", ""])
    eval_sheet.append([""] * 9)
    eval_sheet.append([""] * 9)


In [30]:
eval_df = pd.DataFrame(eval_sheet)

In [31]:
eval_df[:20]

,0,1,2,3,4,5,6,7,8
0,Table,,Exemplar,Generations,Factual Consistency,Style Embodiment,Fluency,ID,1008
1,Name,The Golden Palace,The Golden Curry is a family friendly Indian r...,The Golden Palace is a cheap Italian coffee sh...,,,,0,
2,Eat type,coffee shop,,The Golden Palace is a cheap Italian coffee sh...,,,,2,
3,Food,Italian,,The Golden Palace is a family friendly Italian...,,,,1,
4,Price range,cheap,,,,,,,
5,Customer rating,5 out of 5,,,,,,,
6,Area,riverside,,,,,,,
7,,,,,,,,,
8,,,,,,,,,
9,Table,,Exemplar,Generations,Factual Consistency,Style Embodiment,Fluency,ID,706


In [32]:
eval_df.to_csv("e2e_human_eval_3.csv", index=False, header=False)